***AVISO**: Esse Notebook foi feito com base na estrutura do Databricks Free Edition, que utiliza catálogos.*

# **ETAPA 4 - PROCESSAMENTO PARA CAMADA SILVER**

---
---

<br>

Essa etapa será responsável por mover os dados para a camada silver, limpando e filtrandoos dados.

*`Complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br><br>

---
---

### Parte 1 - **Importação das Bibliotecas Necessárias**

In [0]:
import gc
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

### Parte 2 - **Otimizar a Sessão com configurações Personalizadas**

Aqui o será configurado algumas propriedades para que o desempenho da sessão seja mais otimizado 
- Define tamanho fixo de partições para o shuffle para melhorar o paralelismo (usar ***número de partições = número de núclos de CPU * 2 ou 3*** para encontrar melhor cenário possível)
- Define o tamanho máximo de partições para evitar muitos arquivos pequenos
- Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita
- Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados
- Habilita o Optimize Write do Delta Lake, que reorganiza os dados durante a escrita para gerar arquivos mais balanceados
- Habilita o Auto Compaction do Delta Lake, que consolida automaticamente arquivos pequenos em arquivos maiores após a escrita

In [0]:
spark = (
    SparkSession.builder
        .appName("Silver Layer - Transformations")
        .config("spark.sql.shuffle.partitions", "200")
        .config("spark.sql.files.maxPartitionBytes", "134217728") 
        .config("spark.sql.parquet.compression.codec", "snappy")
        .config("spark.sql.adaptive.enabled", "true")
        .config("spark.databricks.delta.optimizeWrite.enabled", "true")
        .config("spark.databricks.delta.autoCompact.enabled", "true")
        .getOrCreate()
)

### Parte 3 - **Definindo Origens, Arquivos e Destinos**

`Insira nas variáveis:` <br>
- `nome_datalakehouse --> nome do Data Lakehouse `
- `nome_camada_bronze --> nome da camada de origem dos dados`
- `nome_volume_bronze --> nome do volume de origem dos dados dentro da camada`
- `nome_camada_silver --> nome da camada de destino dos dados`
- `nome_volume_silver --> nome do volume de destino dos dados dentro da camada`

In [0]:
nome_datalakehouse = "dataexperts"

nome_camada_bronze = "bronze"
nome_volume_bronze = "vendas_atual" 

nome_camada_silver = "silver"
nome_volume_silver = "vendas" 

O código a seguir armazena em variáveis os caminhos já prontos de origem e de destino dos dados:

In [0]:
origem_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_bronze}/{nome_volume_bronze}"
destino_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_silver}/{nome_volume_silver}"

O código a seguir cria o volume de destino caso ele ainda não exista:

In [0]:
spark.sql(f"CREATE VOLUME IF NOT EXISTS dataexperts.silver.vendas")

`Insira no vetor todos os arquivos que deseja fazer esse processamento para camada bronze:` <br>

In [0]:
bronze_tables = {
    "dim_categoria": "bronze_dim_categoria_produto",
    "dim_cliente": "bronze_dim_cliente",        
    "dim_data": "bronze_dim_data",
    "dim_localidade": "bronze_dim_localidade",                
    "dim_produto": "bronze_dim_produto",                    
    "fato_vendas": "bronze_fato_vendas"                                 
}

bronze_dfs = {                            
    nome: spark.read.format("delta").load(f"{origem_dados}/{tabela}")
    for nome, tabela in bronze_tables.items()
}

### Parte 4 - **Limpando e Filtrando os Dados**

A seguir código para substituir os valores null por N/A:

In [0]:
def replace_null_string(df):
    for coluna, tipo in df.dtypes:
        if tipo == "string" and not coluna.endswith("_id"):
            df = df.withColumn(                                            
                coluna,
                when(col(coluna).isNull(), lit("N/A")).otherwise(col(coluna))
            )
    return df

A seguir código para adicionar SK's e FK's:

In [0]:
def add_surrogate_key(df, sk_name):
    window = Window.orderBy(lit(1))
    return df.withColumn(sk_name, row_number().over(window).cast("long"))

A seguir código para transformar as dimensões:

In [0]:
silver_dim_categoria = (
    bronze_dfs["dim_categoria"]
        .dropDuplicates(["categoria_id"])
        .withColumn("categoria_nome", upper(trim(col("categoria_nome"))))
)
silver_dim_categoria = replace_null_string(silver_dim_categoria)
silver_dim_categoria = add_surrogate_key(silver_dim_categoria, "sk_categoria")

silver_dim_cliente = (
    bronze_dfs["dim_cliente"]
        .dropDuplicates(["cliente_id"])
        .withColumn("nome_cliente", initcap(trim(col("nome_cliente"))))
        .withColumn("estado", upper(col("estado")))
        .withColumn("cidade", initcap(col("cidade")))
)
silver_dim_cliente = replace_null_string(silver_dim_cliente)
silver_dim_cliente = add_surrogate_key(silver_dim_cliente, "sk_cliente")

silver_dim_data = (
    bronze_dfs["dim_data"]
        .dropDuplicates(["data_id"])
)
silver_dim_data = replace_null_string(silver_dim_data)
silver_dim_data = add_surrogate_key(silver_dim_data, "sk_data")

silver_dim_localidade = (
    bronze_dfs["dim_localidade"]
        .dropDuplicates(["localidade_id"])
        .withColumn("estado", upper(col("estado")))
        .withColumn("cidade", initcap(col("cidade")))
)
silver_dim_localidade = replace_null_string(silver_dim_localidade)
silver_dim_localidade = add_surrogate_key(silver_dim_localidade, "sk_localidade")

silver_dim_produto = (
    bronze_dfs["dim_produto"]
        .dropDuplicates(["produto_id"])
        .withColumn("categoria_nome", upper(col("categoria_nome")))
)
silver_dim_produto = replace_null_string(silver_dim_produto)
silver_dim_produto = add_surrogate_key(silver_dim_produto, "sk_produto")

A seguir código para transformar o fato:

In [0]:
silver_fato_vendas = (
    bronze_dfs["fato_vendas"]
        .dropDuplicates(["venda_id"])
        .filter(col("quantidade") > 0)
        # Cliente
        .join(
            silver_dim_cliente.select("cliente_id", "sk_cliente"),
            on="cliente_id",
            how="left"
        )
        # Produto
        .join(
            silver_dim_produto.select("produto_id", "sk_produto"),
            on="produto_id",
            how="left"
        )
        # Data
        .join(
            silver_dim_data.select("data_id", "sk_data"),
            on="data_id",
            how="left"
        )
        # Localidade
        .join(
            silver_dim_localidade.select("localidade_id", "sk_localidade"),
            on="localidade_id",
            how="left"
        )
)

# Removendo FKs null
silver_fato_vendas = silver_fato_vendas.filter(
    col("sk_cliente").isNotNull() &
    col("sk_produto").isNotNull() &
    col("sk_data").isNotNull() &
    col("sk_localidade").isNotNull()
)

# Removendo id's
silver_fato_vendas = silver_fato_vendas.drop(
    "cliente_id",
    "produto_id",
    "data_id",
    "localidade_id"
)

### Parte 5 - **Salvando na Camada Silver**

A seguir, código que salva todas as modificações e os novos arquivos na camada silver:

In [0]:
silver_tables = {
    "silver_dim_categoria_produto": silver_dim_categoria,
    "silver_dim_cliente": silver_dim_cliente,
    "silver_dim_data": silver_dim_data,
    "silver_dim_localidade": silver_dim_localidade,
    "silver_dim_produto": silver_dim_produto,
    "silver_fato_vendas": silver_fato_vendas
}

for nome_tabela, df in silver_tables.items():
    df.write \
        .format("delta") \
        .mode("overwrite") \
        .save(f"{destino_dados}/{nome_tabela}")

### Parte 6 - **Limpeza de Cache e Outros**

O código a seguir exclui todos os Data Frames que podem estar em cache no algoritmo:

In [0]:
   
gc.collect()


<br>

---
---

### **Resultado Final**

Aqui é um log simples para mostrar se deu certo ou não toda a construção da arquitetura planejada.

In [0]:
%skip
silver_dim_categoria_produto = spark.read.format("delta").load("/Volumes/dataexperts/silver/vendas/silver_dim_categoria_produto")
display(silver_dim_categoria_produto)

---
---

<br><br><br><br><br>